In [2]:
pip install librosa

  Created wheel for audioread: filename=audioread-3.0.0-py3-none-any.whl size=23706 sha256=55ab9efa4417ce476f666de592bd270c805891c6459b4b2e45bfdd4d78b69ca8
  Stored in directory: c:\users\valif\appdata\local\pip\cache\wheels\e4\76\a4\cfb55573167a1f5bde7d7a348e95e509c64b2c3e8f921932c3
Successfully built audioread
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install awkward

Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
import numpy as np
import pathlib
import awkward as ak
import librosa

import matplotlib.pyplot as plt

In [9]:
dataset_folder = pathlib.Path("C:\\Users\\valif\\OneDrive\\Desktop\\VScode\\DM2-project\\dm2_timeseries\\RavdessAudioOnly")

In [10]:
file_list = list(dataset_folder.iterdir())
for i, path in enumerate(file_list):
    if path.suffix != ".wav":
        file_list.pop(i)
file_list = sorted(file_list)
file_list

[WindowsPath('C:/Users/valif/OneDrive/Desktop/VScode/DM2-project/dm2_timeseries/RavdessAudioOnly/03-01-01-01-01-01-01.wav'),
 WindowsPath('C:/Users/valif/OneDrive/Desktop/VScode/DM2-project/dm2_timeseries/RavdessAudioOnly/03-01-01-01-01-01-02.wav'),
 WindowsPath('C:/Users/valif/OneDrive/Desktop/VScode/DM2-project/dm2_timeseries/RavdessAudioOnly/03-01-01-01-01-01-03.wav'),
 WindowsPath('C:/Users/valif/OneDrive/Desktop/VScode/DM2-project/dm2_timeseries/RavdessAudioOnly/03-01-01-01-01-01-04.wav'),
 WindowsPath('C:/Users/valif/OneDrive/Desktop/VScode/DM2-project/dm2_timeseries/RavdessAudioOnly/03-01-01-01-01-01-05.wav'),
 WindowsPath('C:/Users/valif/OneDrive/Desktop/VScode/DM2-project/dm2_timeseries/RavdessAudioOnly/03-01-01-01-01-01-06.wav'),
 WindowsPath('C:/Users/valif/OneDrive/Desktop/VScode/DM2-project/dm2_timeseries/RavdessAudioOnly/03-01-01-01-01-01-07.wav'),
 WindowsPath('C:/Users/valif/OneDrive/Desktop/VScode/DM2-project/dm2_timeseries/RavdessAudioOnly/03-01-01-01-01-01-08.wav'),


In [11]:
array = list()
for path in file_list:
    librosa_audio_segment, sr = librosa.load(path, sr=None)
    array.append([librosa_audio_segment])

In [16]:
b = ak.ArrayBuilder()
for path in file_list:
    librosa_audio_segment, sr = librosa.load(path, sr=None)
    b.begin_list()
    b.begin_list()
    for value in librosa_audio_segment:
        b.real(value)
    b.end_list() 
    b.end_list() 
array = b.snapshot()

In [17]:
CATEGORICAL_FEATURES_NAMES = {
    "modality": {"01": "full-AV", "02": "video-only", "03": "audio-only"}, 
    "vocal_channel": {"01": "speech", "02": "song"},
    "emotion": {"01" : "neutral", "02" : "calm", "03" : "happy", "04" : "sad", "05" : "angry", "06" : "fearful", "07" : "disgust", "08" : "surprised"},
    "emotional_intensity": {"01" : "normal", "02" : "strong"},
    "statement": {"01" : "Kids are talking by the door", "02" : "Dogs are sitting by the door"},
    "repetition": {"01" : "1st", "02" : "2nd"},
    "actor": {str(i).zfill(2): str(i).zfill(2) for i in range(1, 25)}
}

In [18]:
df = list()
file_names = list()
for path in file_list:
    stem = path.stem.split("-")
    file_names.append(path.name)
    df.append(stem)
df = pd.DataFrame(df)
df.columns = list(CATEGORICAL_FEATURES_NAMES.keys())

for column in df.columns:
    df = df.replace(CATEGORICAL_FEATURES_NAMES)

df["sex"] = ["F" if i % 2 == 0 else "M" for i in df["actor"].astype(int)]
df["filename"] = file_names

In [19]:
df.head()

,modality,vocal_channel,emotion,emotional_intensity,statement,repetition,actor,sex,filename
0,audio-only,speech,neutral,normal,Kids are talking by the door,1st,01,M,03-01-01-01-01-01-01.wav
1,audio-only,speech,neutral,normal,Kids are talking by the door,1st,02,F,03-01-01-01-01-01-02.wav
2,audio-only,speech,neutral,normal,Kids are talking by the door,1st,03,M,03-01-01-01-01-01-03.wav
3,audio-only,speech,neutral,normal,Kids are talking by the door,1st,04,F,03-01-01-01-01-01-04.wav
4,audio-only,speech,neutral,normal,Kids are talking by the door,1st,05,M,03-01-01-01-01-01-05.wav


In [20]:
df_sorted_actor = df.sort_values(by=["actor", "filename"])
idxs_sorted = list(df_sorted_actor.index)

In [21]:
idx_train = list(df_sorted_actor[df_sorted_actor.actor.astype(int) < 19].index)
idx_test = list(df_sorted_actor[df_sorted_actor.actor.astype(int) >= 19].index)

In [22]:
X_train = array[idx_train]
X_test = array[idx_test]
Y_train = df_sorted_actor[df_sorted_actor.actor.astype(int) < 19]
Y_test = df_sorted_actor[df_sorted_actor.actor.astype(int) >= 19]

In [31]:
def pad_X(X, m_max, nan_value=0):
    return ak.fill_none(ak.pad_none(X, m_max, axis=2, clip=True), value=nan_value)

In [35]:
# find the max length of X_train
maximum = 0
for ts in X_train:
    length = len(np.asarray(np.ravel(ts)))
    if length > maximum:
        maximum = length
maximum

304304

In [38]:
def pad_time_series(time_series_list,max, padding_value=0):
   

    # Pad all time series to the length of the longest time series
    padded_time_series_list = [
        np.pad(ts, (0, max - len(ts)), constant_values=padding_value)
        for ts in time_series_list
    ]

    return padded_time_series_list

In [ ]:
X_train = np.squeeze(np.array(pad_X(X_train, maximum, 0)))
X_test = np.squeeze(np.array(pad_X(X_test, maximum, 0)))

In [51]:
X_train1=ak.Array(X_train)

In [ ]:
X_train2 = np.squeeze(np.array(pad_X(X_train1, maximum, 0)))